In [1]:
# New notebook for BERT until it works properly.

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_pickle("./data/cleaned_submissions.pkl")
data = data.loc[data["sentiment"] != "", ["text", "sentiment"]] # Only get labeled instances

# Encode labels
lenc = LabelEncoder()
y = lenc.fit_transform(data["sentiment"])

# Vectorize text using tfidf
tfidf = TfidfVectorizer(preprocessor=' '.join, lowercase=False, min_df=5) # min_df = Minimum occurance of words
X = tfidf.fit_transform(data["text"])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
model.summary()

2021-11-22 17:14:53.065611: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Data

# DEV!!!
# Less data, so I can check if the model works
X_train_bert = X_train[:50] # X_train.toarray()[:50, :, None] -> from lstm
y_train_bert = y_train[:50]

In [29]:
import numpy as np
from tensorflow.sparse import reorder
import tensorflow as tf

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, values = coo.data, dense_shape = coo.shape)

In [54]:
def convert_sparse_matrix_to_dictionary(X):
    dic = {}
    coo = X.tocoo()
    for i in range(0, coo.row):
        dic[i] = item.col
    print(dic)

In [31]:
coo = X_train_bert.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
print(tf.SparseTensor(indices, coo.data, coo.shape))

SparseTensor(indices=tf.Tensor(
[[   0 3057]
 [   0 2642]
 [   0 3787]
 ...
 [  48 2746]
 [  48 2137]
 [  49 2137]], shape=(669, 2), dtype=int64), values=tf.Tensor(
[0.59542567 0.60564568 0.51284646 0.1251003  0.62899846 0.74640758
 0.21733997 0.15881666 0.15065584 0.14706971 0.12751685 0.14938152
 0.12992945 0.14706971 0.16099216 0.16099216 0.12868378 0.11051738
 0.12347085 0.12950483 0.10531676 0.12173798 0.10163091 0.13373161
 0.52506096 0.14818947 0.09749081 0.10389945 0.08722882 0.09184639
 0.31873883 0.08660123 0.09368347 0.06484686 0.11916851 0.07623556
 0.09338411 0.22905329 0.09690672 0.09801749 0.04434515 0.08364269
 0.07285628 0.10389945 0.08684972 0.10210092 0.07692903 0.09493194
 0.07525298 0.11471555 0.07061359 0.11990253 0.06940748 0.07790467
 0.08529129 0.06971818 0.0898785  0.05564887 0.48529515 0.41759496
 0.36845519 0.29808807 0.37597191 0.40961901 0.22544147 0.07430258
 0.65422268 0.48853088 0.56989251 0.09248129 0.41630068 0.37163055
 0.81454622 0.15842644 0.396248

In [32]:
X_train_berti = reorder(convert_sparse_matrix_to_sparse_tensor(X_train_bert))

In [38]:
type(X_train_berti)

tensorflow.python.framework.sparse_tensor.SparseTensor

In [37]:
tf.sparse.to_dense(X_train_berti, validate_indices = False)

<tf.Tensor: shape=(50, 4936), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [35]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(X_train_berti, y_train_bert,  epochs = 2)

Epoch 1/2


ValueError: in user code:

    File "/Users/stefanwinter/Local/Thesis/App/.thesis/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/stefanwinter/Local/Thesis/App/.thesis/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/stefanwinter/Local/Thesis/App/.thesis/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/stefanwinter/Local/Thesis/App/.thesis/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 796, in train_step
        y_pred = self(x, training=True)
    File "/Users/stefanwinter/Local/Thesis/App/.thesis/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: E

In [78]:
# Implementing BERT from scratch, without tf-idf data -> will use BERT tokenizer

import pandas as pd
data = pd.read_pickle("./data/cleaned_submissions.pkl")
data = data.loc[data["sentiment"] != "", ["text", "sentiment"]] # Only get labeled instances
text = data["text"][:50]
labels = data["sentiment"][:50]

In [70]:
max_len = max([len(sent) for sent in text])
print('Max length: ', max_len)

Max length:  370


In [87]:
from transformers import BertTokenizer
import re
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=372,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [80]:
X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2020)

In [91]:
# Concatenate train data and test data
all_posts = np.concatenate([X_train, X_val])

# Encode our concatenated data
encoded_posts = [tokenizer.encode(sent, add_special_tokens=True) for sent in text]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_posts])
print('Max length: ', max_len)

Max length:  372


In [93]:
# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

TypeError: expected string or bytes-like object

In [94]:
# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Tokenizing data...


TypeError: expected string or bytes-like object